In [90]:
#If you haven't run the following commands in comments before, run them by removing the # 
#jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [91]:
#jupyter serverextension enable voila --sys-prefix

Enabling: voila
- Writing config: C:\Users\nikhi\anaconda3\envs\webapp-venv\etc\jupyter
    - Validating...
      voila 0.2.15 ok


In [32]:
#!pip install scikit-learn

In [33]:
#!pip install pandas

In [34]:
#!pip install pipenv

In [35]:
#!pip install pandas_datareader

In [36]:
#!pip install matplotlib

In [37]:
#!pip install xgboost

In [1]:
from IPython.core.debugger import set_trace
from sklearn.model_selection import train_test_split
import pandas as pd
import pandas_datareader as web
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import xgboost as xgb
%matplotlib inline

In [2]:
import warnings
import ipywidgets as widgets
from IPython.display import display, clear_output




warnings.filterwarnings('ignore')

In [3]:
file = open("image.jpg", "rb")
image = file.read()
image_headline = widgets.Image(
    value = image,
    format = 'jpg',
    width = '300'
)
label_headline = widgets.Label(
    value = 'Photo from kiplinger.com',
    style = {'description_width': 'initial'}
)
vbox_headline = widgets.VBox([image_headline, label_headline])

In [4]:
companyName = widgets.Text(placeholder = 'Enter the NASDAQ ticker symbol of the company you want to analyse')

In [5]:
date1 = widgets.DatePicker(description = 'From date')
date2 = widgets.DatePicker(description = 'To date')

In [6]:
button_send = widgets.Button(
    description = 'ANALYSE',
    tooltip = 'Send',
    style = {'description_width': 'initial'}
)
output = widgets.Output()

In [7]:
def actualData(y_test):
    arr = pd.DataFrame(y_test).to_numpy()
    plt.figure(figsize = (16, 8))
    plt.title('Actual Data')
    plt.xlabel('Time', fontsize = 18)
    plt.ylabel('Close Price USD ($)', fontsize = 18)
    plt.plot(arr)
    plt.show()

In [8]:
def plotPredictedData(y_test, predictions):
    arr = pd.DataFrame(y_test).to_numpy()
    plt.figure(figsize = (16, 8))
    plt.title('Predicted Data depicted in Orange Color')
    plt.xlabel('Time', fontsize = 18)
    plt.ylabel('Close Price USD ($)', fontsize = 18)
    plt.plot(arr)
    plt.plot(predictions)
    plt.show()

In [9]:
def predict(companyName, date1, date2):
    #df and df0 contains the dataset of the stock.
    df = web.DataReader(companyName, data_source = 'yahoo', start = date1, end = date2)
    df0 = web.DataReader('AAPL', data_source = 'yahoo', start = date1, end = date2)
    
    df["target"] = df.Close.shift(-1)
    
    df.dropna(inplace = True)
    #The target: Consists of the day after's closed stock price 
    target = df["target"]
    target = target.append(df0["Close"].tail(1))
    #Here the data is being split into testing and training sets. 80% of the data used for training and 20% of the data is
    #used for testing 
    X_train, X_test, y_train, y_test = train_test_split(df0, target, test_size = 0.2, random_state = 0)
    X_test = X_test.append(df0.tail(1))
    y_test = y_test.append(target.tail(1))
    
    train = xgb.DMatrix(X_train, label = y_train)
    test = xgb.DMatrix(X_test, label = y_test)
    
    param = {
    'max_depth': 4,
    'eta': 0.3,
    'objective': 'reg:squarederror',
    'num_class': 1
    }
    epochs = 20
    
    model = xgb.train(param, train, epochs)
    
    predictions = model.predict(test)
    #Tommorow's prediction
    print(predictions[len(predictions) - 1])

In [10]:
def plotGraphs(companyName, date1, date2):
    #df and df0 contains the dataset of the stock.
    df = web.DataReader(companyName, data_source = 'yahoo', start = date1, end = date2)
    
    df["target"] = df.Close.shift(-1)
    
    df.dropna(inplace = True)
    #The target: Consists of the day after's closed stock price
    target = df["target"]
    df.drop(["target"], axis = 1, inplace = True)
    #Here the data is being split into testing and training sets. 80% of the data used for training and 20% of the data is
    #used for testing
    X_train, X_test, y_train, y_test = train_test_split(df, target, test_size = 0.2, random_state = 0)
    
    train = xgb.DMatrix(X_train, label = y_train)
    test = xgb.DMatrix(X_test, label = y_test)
    
    param = {
    'max_depth': 4,
    'eta': 0.3,
    'objective': 'reg:squarederror',
    'num_class': 1
    }
    epochs = 20
    
    model = xgb.train(param, train, epochs)
    
    predictions = model.predict(test)
    #Plotting Data
    actualData(y_test)
    plotPredictedData(y_test, predictions)

In [11]:
def on_button_clicked(event):
    with output:
        clear_output()
        print("Prediction for tommorow: ")
        predict(companyName.value, date1.value, date2.value)
        plotGraphs(companyName.value, date1.value, date2.value)

        
button_send.on_click(on_button_clicked)
vbox_result = widgets.VBox([button_send, output])

In [12]:
text_0 = widgets.HTML(value = "<h1>AI Stock Market Analyst</h1>")
text_1 = widgets.HTML(value = "<h2>First enter the NASDAQ ticker symbol of the company you want to analyse.</h2>")
text_2 = widgets.HTML(value = "<h2>Enter the timeframe of data fed into the machine learning algorithm.</h2>")
text_3 = widgets.HTML(value = "<h3>The prediction for tommorow is the day after the to date.</h3>")
vbox_text = widgets.VBox([text_0, text_1, companyName, text_2, text_3, date1, date2, vbox_result])

In [13]:
page = widgets.HBox([vbox_headline, vbox_text])
display(page)

In [14]:
!pip freeze > requirements.txt